# Flower Federated Learning with Placeholder Clients

This notebook demonstrates federated learning using Flower's built-in placeholder clients instead of custom implementations. This approach is simpler and shows the core federated averaging concept without custom client code.


In [1]:
# Install required dependencies
import subprocess
import sys

def install_package(package):
    try:
        __import__(package)
        print(f"✅ {package} is already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installed successfully")

# List of required packages
required_packages = [
    "flwr[simulation]",
    "torch",
    "torchvision",
    "numpy"
]

print("🔍 Checking and installing dependencies...")
for package in required_packages:
    install_package(package)

print("\n🎉 All dependencies are ready!")


🔍 Checking and installing dependencies...
📦 Installing flwr[simulation]...



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


✅ flwr[simulation] installed successfully
✅ torch is already installed
✅ torchvision is already installed
✅ numpy is already installed

🎉 All dependencies are ready!


In [2]:
# Import required libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
from typing import List

import flwr
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr.common import ndarrays_to_parameters, Context

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")


/Users/kevincorstorphine/.local/share/virtualenvs/ai.llm_prompting-rP8wfVkF/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-05 12:00:24,035	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Training on cpu
Flower 1.20.0 / PyTorch 2.2.2


In [3]:
# Define a simple neural network architecture
class SimpleNet(nn.Module):
    def __init__(self) -> None:
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Utility functions for parameter handling
def get_parameters(net) -> List[np.ndarray]:
    """Get model parameters as a list of NumPy arrays."""
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    """Set model parameters from a list of NumPy arrays."""
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

# Create initial model and get parameters
net = SimpleNet().to(DEVICE)
initial_parameters = get_parameters(net)
print(f"Model created with {len(initial_parameters)} parameter tensors")
print(f"Total parameters: {sum(p.size for p in initial_parameters):,}")


Model created with 10 parameter tensors
Total parameters: 62,006


In [4]:
# Configuration
NUM_CLIENTS = 10
NUM_ROUNDS = 5

print(f"Federated Learning Configuration:")
print(f"- Number of clients: {NUM_CLIENTS}")
print(f"- Number of rounds: {NUM_ROUNDS}")
print(f"- Device: {DEVICE}")


Federated Learning Configuration:
- Number of clients: 10
- Number of rounds: 5
- Device: cpu


In [5]:
# Server function that creates the federated learning strategy
def server_fn(context: Context) -> ServerAppComponents:
    """Create server components with FedAvg strategy."""
    
    # Create FedAvg strategy with federated averaging
    strategy = FedAvg(
        fraction_fit=1.0,  # Use 100% of available clients for training
        fraction_evaluate=0.5,  # Use 50% of available clients for evaluation
        min_fit_clients=NUM_CLIENTS,  # Require all clients for training
        min_evaluate_clients=5,  # Require at least 5 clients for evaluation
        min_available_clients=NUM_CLIENTS,  # Wait for all clients to be available
        initial_parameters=ndarrays_to_parameters(initial_parameters),  # Start with initial model
    )
    
    # Configure server for specified number of rounds
    config = ServerConfig(num_rounds=NUM_ROUNDS)
    
    return ServerAppComponents(strategy=strategy, config=config)

# Create the ServerApp
server = ServerApp(server_fn=server_fn)
print("✅ Server created with FedAvg strategy")


✅ Server created with FedAvg strategy


In [6]:
# Configure client resources
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# If using GPU, allocate one GPU per client
if DEVICE == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}

print(f"Backend configuration: {backend_config}")


Backend configuration: {'client_resources': {'num_cpus': 1, 'num_gpus': 0.0}}


In [ ]:
# Run the federated learning simulation
print("🚀 Starting federated learning simulation...")
print("\nThis will demonstrate:")
print("1. Server sends global model to all clients")
print("2. Each client trains on their local data")
print("3. Server aggregates (averages) all client models")
print("4. Process repeats for multiple rounds")
print("\nNote: Using placeholder clients means no actual training occurs,")
print("but the federated averaging mechanism is demonstrated.")

# Run simulation with placeholder clients
run_simulation(
    server_app=server,
    client_app=None,  # No custom client app - uses placeholder clients
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)


INFO :      Starting Flower ServerApp, config: num_rounds=5, no round_timeout
ERROR :     An exception occurred !! Either `client_app_attr` or `client_app` must be provided
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
ERROR :     Traceback (most recent call last):
  File "/Users/kevincorstorphine/.local/share/virtualenvs/ai.llm_prompting-rP8wfVkF/lib/python3.11/site-packages/flwr/simulation/run_simulation.py", line 387, in _main_loop
    vce.start_vce(
  File "/Users/kevincorstorphine/.local/share/virtualenvs/ai.llm_prompting-rP8wfVkF/lib/python3.11/site-packages/flwr/server/superlink/fleet/vce/vce_api.py", line 390, in start_vce
    raise ex
  File "/Users/kevincorstorphine/.local/share/virtualenvs/ai.llm_prompting-rP8wfVkF/lib/python3.11/site-packages/flwr/server/superlink/fleet/vce/vce_api.py", line 359, in start_vce
    client_app = app_fn()
                 ^^^^^^^^
  F

🚀 Starting federated learning simulation...

This will demonstrate:
1. Server sends global model to all clients
2. Each client trains on their local data
3. Server aggregates (averages) all client models
4. Process repeats for multiple rounds

Note: Using placeholder clients means no actual training occurs,
but the federated averaging mechanism is demonstrated.


In [ ]:
# Demonstrate the federated averaging concept
print("\n📊 Federated Averaging Demonstration:")
print("\nIn each round:")
print("1. Server has global model G_round")
print("2. All clients receive G_round")
print("3. Each client trains G_round on their local data → L_client")
print("4. Server computes: G_round+1 = average(L_client1, L_client2, ..., L_clientN)")
print("5. Next round starts with G_round+1")

print("\n🔑 Key Points:")
print("- Each client starts each round with the SAME global model")
print("- The global model is the federated average of all clients from previous round")
print("- This ensures all clients benefit from the collective learning")
print("- No client's data is ever shared, only model parameters")

print("\n✅ Federated averaging is working when you see:")
print("- 'aggregate_fit: received X results and 0 failures' in the logs")
print("- Multiple rounds completing successfully")
print("- All clients participating in each round")


In [ ]:
# Show how to access the final global model
print("\n🎯 Accessing the Final Global Model:")
print("\nAfter the simulation completes, the server holds the final global model.")
print("This model represents the federated average of all client models")
print("after all training rounds.")

print("\nTo use the final model in your own code:")
print("1. The server's strategy contains the final parameters")
print("2. You can extract them and load into your model")
print("3. This model has learned from all clients' data without seeing it directly")

# Example of how you would load the final model
print("\nExample code to load final model:")
print("""
# After simulation completes
final_model = SimpleNet().to(DEVICE)
# final_parameters = server.strategy.parameters  # Get final parameters
# set_parameters(final_model, final_parameters)  # Load into model
# Now final_model contains the federated average of all clients
""")
